In [3]:
from op_analytics.coreutils.duckdb_inmem.client import init_client
from op_analytics.cli.subcommands.pulls.growthepie.dataaccess import GrowThePie

duckdb_ctx = init_client()

In [5]:
fundamentals = GrowThePie.FUNDAMENTALS_SUMMARY.read(min_date="2024-11-01")

2025-01-03 05:58:28 [info     ] querying markers for 'growthepie/chains_daily_fundamentals_v1' DateFilter(min_date=datetime.date(2024, 11, 1), max_date=None, datevals=None) filename=dailydata.py lineno=112 process=98461
2025-01-03 05:58:29 [info     ] 65 markers found               filename=dailydata.py lineno=126 process=98461
2025-01-03 05:58:29 [info     ] 62 distinct paths              filename=dailydata.py lineno=132 process=98461
2025-01-03 05:58:32 [info     ] registered view: 'growthepie_chains_daily_fundamentals_v1' using 62 parquet paths filename=client.py lineno=159 process=98461
┌─────────────────────────────────────────┐
│                  name                   │
│                 varchar                 │
├─────────────────────────────────────────┤
│ growthepie_chains_daily_fundamentals_v1 │
└─────────────────────────────────────────┘



In [6]:
duckdb_ctx.client.sql(f"DESCRIBE {fundamentals}")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ metric_key  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ origin_key  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ value       │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ dt          │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [7]:
duckdb_ctx.client.sql(f"SELECT * FROM {fundamentals} LIMIT 10")

┌────────────────┬────────────┬────────────────────┬────────────┐
│   metric_key   │ origin_key │       value        │     dt     │
│    varchar     │  varchar   │       double       │    date    │
├────────────────┼────────────┼────────────────────┼────────────┤
│ daa            │ starknet   │             7245.0 │ 2024-11-01 │
│ txcount        │ rhino      │             4426.0 │ 2024-11-01 │
│ daa            │ rhino      │             1641.0 │ 2024-11-01 │
│ txcount        │ starknet   │            59311.0 │ 2024-11-01 │
│ txcount        │ blast      │           513018.0 │ 2024-11-01 │
│ daa            │ blast      │            19741.0 │ 2024-11-01 │
│ aa_last7d      │ orderly    │               55.0 │ 2024-11-01 │
│ txcount        │ worldchain │           520595.0 │ 2024-11-01 │
│ gas_per_second │ blast      │ 2.4999156911805556 │ 2024-11-01 │
│ txcount        │ gravity    │          2003045.0 │ 2024-11-01 │
├────────────────┴────────────┴────────────────────┴────────────┤
│ 10 rows 